In [1]:
import pandas as pd

# Modules import
import preprocessing as pp
import model
from train import train_model
from predict import predict

In [2]:
# Data loading
df_store = pd.read_csv('../data/store.csv')
df_train = pd.read_csv('../data/train.csv', low_memory=False)
df_test = pd.read_csv('../data/test.csv', low_memory=False)

## Coherence between train and test sets

Tell whether train and test data follow the same distribution

## General pipeline test

### Model selection

In [3]:
# Preprocessing
X, y = pp.Preprocessor().transform(df_store, df_train)
print(X.shape, y.shape)
X.head()

D:\Etudes\Cours\ING5\Modules\S2\ML Business Case\0. Repository\model\preprocessing.py:37: DataConversionWarning: Data with input dtype float32 were all converted to float64 by the scale function.
  df_store_bis = scale(df_store_bis)


(1017209, 12) (1017209,)


,DayOfWeek,Open,Promo,SchoolHoliday,StateHoliday_a,StateHoliday_b,StateHoliday_c,cos_DayOfWeek,sin_DayOfWeek,1Component Store PCA,2Component Store PCA,3Component Store PCA
0,5.0,1.0,1.0,1.0,0.0,0.0,0.0,-0.222521,-0.974928,0.953691,-0.744872,-2.383955
1,4.0,1.0,1.0,1.0,0.0,0.0,0.0,-0.900969,-0.433884,0.953691,-0.744872,-2.383955
2,3.0,1.0,1.0,1.0,0.0,0.0,0.0,-0.900969,0.433884,0.953691,-0.744872,-2.383955
3,2.0,1.0,1.0,1.0,0.0,0.0,0.0,-0.222521,0.974928,0.953691,-0.744872,-2.383955
4,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.623490,0.781832,0.953691,-0.744872,-2.383955


In [4]:
reg_model = model.Regressor()
train_model(reg_model, X, y)

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 2 folds for each of 18 candidates, totalling 36 fits
[CV] learning_rate=0.01, max_depth=3, n_estimators=10 ................
[CV]  learning_rate=0.01, max_depth=3, n_estimators=10, score=-1.7680222800906655, total=   4.1s
[CV] learning_rate=0.01, max_depth=3, n_estimators=10 ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.3s remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=3, n_estimators=10, score=-1.7075608902762416, total=   2.9s
[CV] learning_rate=0.01, max_depth=3, n_estimators=100 ...............


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    7.5s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  2.3min remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=10, n_estimators=10, score=-1.7642693703602785, total=   6.7s
[CV] learning_rate=0.01, max_depth=10, n_estimators=10 ...............


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.4min remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=10, n_estimators=10, score=-1.7103990088886079, total=   6.7s
[CV] learning_rate=0.01, max_depth=10, n_estimators=100 ..............
[CV]  learning_rate=0.01, max_depth=10, n_estimators=100, score=0.1509074555290022, total= 1.1min
[CV] learning_rate=0.01, max_depth=10, n_estimators=100 ..............
[CV]  learning_rate=0.01, max_depth=10, n_estimators=100, score=0.18769822456782936, total= 1.2min
[CV] learning_rate=0.1, max_depth=3, n_estimators=10 .................
[CV]  learning_rate=0.1, max_depth=3, n_estimators=10, score=0.20731327643231834, total=   4.2s
[CV] learning_rate=0.1, max_depth=3, n_estimators=10 .................
[CV]  learning_rate=0.1, max_depth=3, n_estimators=10, score=0.22846065244421054, total=   3.8s
[CV] learning_rate=0.1, max_depth=3, n_estimators=100 ................
[CV]  learning_rate=0.1, max_depth=3, n_estimators=100, score=0.5582185202984948, total=  29.9s
[CV] learning_rate=0.1, max_depth=3, n_estimators=100 ........

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 15.1min finished


In [6]:
print(reg_model)

Regressor()
